Rare diseases

In [35]:
"""1º Importo las librerías que creo que puedo necesitar en esta tarea."""

# Manejo de datos
import pandas as pd  # Análisis y manipulación de datos en tablas
import numpy as np  # Cálculos numéricos y matrices
import os

# Visualización de datos
import matplotlib.pyplot as plt  # Gráficos básicos en 2D
import seaborn as sns  # Gráficos estadísticos mejorados
import plotly.express as px  # Gráficos interactivos simplificados
import plotly.graph_objects as go  # Control avanzado de gráficos en Plotly
import plotly.io as pio  # Interfaz de entrada/salida de Plotly
import squarify  # Visualización de diagramas de "treemap"

# Web Scraping y APIs
import requests  # Peticiones HTTP para obtener datos de APIs como PubChem

# Quimioinformática y Bioinformática
from rdkit import Chem  # Manejo de estructuras químicas
from rdkit.Chem import DataStructs, AllChem  # Análisis molecular
import pubchempy as pcp  # Interacción con la base de datos de PubChem

In [36]:
carpeta_data = r"C:\Users\maria\ITAcademy\Sprint 10 - Pràctica final\Data\tablas_extraidas_Orphadata"
orphadata = {}
for archivo in os.listdir(carpeta_data):
    if archivo.endswith(".csv"):
        ruta_archivo = os.path.join(carpeta_data, archivo)
        nombre_dataframe = archivo.replace(".csv", "")
        orphadata[nombre_dataframe] = pd.read_csv(ruta_archivo)
orphadata.keys()

dict_keys(['df_diseases', 'df_disease_gene', 'df_genes', 'df_rd_classification', 'df_rd_epidemiology', 'df_rd_functional_consequences', 'df_rd_history', 'df_rd_list', 'df_rd_phenotypes'])

In [37]:
# Cambiar nombre col hchId
orphadata['df_diseases'].rename(columns={'class_hchId': 'hchId'}, inplace=True)

# Cambiar nombre col Orphacode
orphadata['df_rd_epidemiology'].rename(columns={'Code Disease': 'ORPHAcode'}, inplace=True)

# cambiar nombre gene name y gene symbol
orphadata['df_genes'].rename(columns={'name': 'Gene name'}, inplace=True)
orphadata['df_genes'].rename(columns={'symbol': 'Gene symbol'}, inplace=True)

# Cambiar nombre col Orphacode
orphadata['df_rd_functional_consequences'].rename(columns={'Código Orphanet': 'ORPHAcode'}, inplace=True)

# Cambiar nombre col pref termm
orphadata['df_rd_functional_consequences'].rename(columns={'Nombre': 'disease_name'}, inplace=True)

In [38]:
for nombre_archivo, dataframe in orphadata.items():
    print(f"Archivo: {nombre_archivo}")
    print(f"Total filas = {len(dataframe)}")
    print("Columnas:", dataframe.columns.tolist())
    print("\n")

Archivo: df_diseases
Total filas = 27613
Columnas: ['hchId', 'ORPHAcode', 'preferredTerm']


Archivo: df_disease_gene
Total filas = 8193
Columnas: ['ORPHAcode', 'Disorder name', 'Disorder type', 'Gene symbol', 'Gene name', 'Gene type', 'Gene synonyms', 'Locus', 'Validation source']


Archivo: df_genes
Total filas = 4478
Columnas: ['HGNC', 'Gene name', 'Gene symbol']


Archivo: df_rd_classification
Total filas = 35
Columnas: ['hchId', 'hchTag']


Archivo: df_rd_epidemiology
Total filas = 16119
Columnas: ['Typology', 'ORPHAcode', 'Preferred term', 'Prevalence Type', 'Prevalence Value']


Archivo: df_rd_functional_consequences
Total filas = 1046
Columnas: ['disease_name', 'ORPHAcode', 'Consecuencias Funcionales']


Archivo: df_rd_history
Total filas = 6994
Columnas: ['data', 'AverageAgeOfOnset', 'Date', 'DisorderGroup', 'ORPHAcode', 'OrphanetURL', 'Preferred term', 'TypeOfInheritance', 'Typology']


Archivo: df_rd_list
Total filas = 11074
Columnas: ['ORPHAcode', 'Preferred term']


Archiv

In [39]:
# Mega merge
tabla_completa = orphadata['df_diseases'] \
    .merge(orphadata['df_disease_gene'], on='ORPHAcode') \
    .merge(orphadata['df_genes'], on='Gene symbol') \
    .merge(orphadata['df_rd_classification'], on='hchId') \
    .merge(orphadata['df_rd_epidemiology'], on='ORPHAcode') \
    .merge(orphadata['df_rd_functional_consequences'], on='ORPHAcode') \
    .merge(orphadata['df_rd_history'], on='ORPHAcode') \
    .merge(orphadata['df_rd_list'], on='ORPHAcode') \
    .merge(orphadata['df_rd_phenotypes'], on='ORPHAcode')
print(tabla_completa.shape)
tabla_completa.columns

(2122829, 33)


Index(['hchId', 'ORPHAcode', 'preferredTerm', 'Disorder name', 'Disorder type',
       'Gene symbol', 'Gene name_x', 'Gene type', 'Gene synonyms', 'Locus',
       'Validation source', 'HGNC', 'Gene name_y', 'hchTag', 'Typology_x',
       'Preferred term_x', 'Prevalence Type', 'Prevalence Value',
       'disease_name', 'Consecuencias Funcionales', 'data',
       'AverageAgeOfOnset', 'Date', 'DisorderGroup', 'OrphanetURL',
       'Preferred term_y', 'TypeOfInheritance', 'Typology_y', 'Preferred term',
       'Disorder', 'HPO_ID', 'HPO_Term', 'Frequency'],
      dtype='object')

In [40]:
tabla_completa.head(10)

,hchId,ORPHAcode,preferredTerm,Disorder name,Disorder type,Gene symbol,Gene name_x,Gene type,Gene synonyms,Locus,...,DisorderGroup,OrphanetURL,Preferred term_y,TypeOfInheritance,Typology_y,Preferred term,Disorder,HPO_ID,HPO_Term,Frequency
0,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0000083,Renal insufficiency,Very frequent (99-80%)
1,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0000100,Nephrotic syndrome,Very frequent (99-80%)
2,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0000365,Hearing impairment,Very frequent (99-80%)
3,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0000524,Conjunctival telangiectasia,Very frequent (99-80%)
4,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0000790,Hematuria,Very frequent (99-80%)
5,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0000962,Hyperkeratosis,Very frequent (99-80%)
6,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0000966,Hypohidrosis,Very frequent (99-80%)
7,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0001014,Angiokeratoma,Very frequent (99-80%)
8,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0001131,Corneal dystrophy,Very frequent (99-80%)
9,146,324,Fabry disease,Fabry disease,Disease,GLA,galactosidase alpha,gene with protein product,['GALA'],"[{'GeneLocus': 'Xq22.1', 'LocusKey': 1}]",...,Disorder,http://www.orpha.net/consor/cgi-bin/OC_Exp.php...,Fabry disease,"['X-linked dominant', 'X-linked recessive']",Disease,Fabry disease,Fabry disease,HP:0001369,Arthritis,Very frequent (99-80%)


In [41]:
tabla_completa.columns.duplicated()     # parece que no hay columnas duplicadas

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [42]:
print(len(tabla_completa['Gene name_x'].unique()))
print(len(tabla_completa['Gene name_y'].unique()))
print("\n")
print(len(tabla_completa['Typology_x'].unique()))
print(len(tabla_completa['Typology_y'].unique()))
print("\n")
print(len(tabla_completa['Preferred term'].unique()))
print(len(tabla_completa['Preferred term_y'].unique()))

1347
1347


8
8


561
561


In [43]:
tabla_completa.drop(columns=['Gene name_y', 'Typology_y', 'Preferred term_y'], inplace=True)

In [44]:
nuevo_orden_cols = []

In [45]:
tabla_completa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2122829 entries, 0 to 2122828
Data columns (total 30 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   hchId                      int64  
 1   ORPHAcode                  int64  
 2   preferredTerm              object 
 3   Disorder name              object 
 4   Disorder type              object 
 5   Gene symbol                object 
 6   Gene name_x                object 
 7   Gene type                  object 
 8   Gene synonyms              object 
 9   Locus                      object 
 10  Validation source          object 
 11  HGNC                       float64
 12  hchTag                     object 
 13  Typology_x                 object 
 14  Preferred term_x           object 
 15  Prevalence Type            object 
 16  Prevalence Value           float64
 17  disease_name               object 
 18  Consecuencias Funcionales  float64
 19  data                       int64  
 20  Av

# En diferentes tablas

In [ ]:
df_enfermedades = orphadata['df_diseases'] \
    .merge(orphadata['df_rd_list'], on='ORPHAcode', how='outer')

df_enfermedades.drop(columns=['Preferred term'], inplace=True)

df_enfermedades = df_enfermedades.drop_duplicates(subset='ORPHAcode')

df_enfermedades

,hchId,ORPHAcode,preferredTerm
0,146.0,5,Long chain 3-hydroxyacyl-CoA dehydrogenase def...
7,150.0,6,3-methylcrotonyl-CoA carboxylase deficiency
9,147.0,7,3C syndrome
12,147.0,8,"47,XYY syndrome"
14,147.0,9,Tetrasomy X syndrome
...,...,...,...
29026,147.0,689408,Shashi-Pena syndrome
29029,147.0,689422,Okur-Chung neurodevelopmental syndrome
29032,197.0,689430,Adenoid ameloblastoma
29034,147.0,689822,Structural heart defects-renal anomalies syndrome


In [55]:
df_enfermedades_clasificacion = df_enfermedades.merge(orphadata['df_rd_classification'], on='hchId', how='left')
df_enfermedades_clasificacion = df_enfermedades_clasificacion[['hchId', 'hchTag', 'ORPHAcode', 'preferredTerm']]
df_enfermedades_clasificacion

,hchId,hchTag,ORPHAcode,preferredTerm
0,146.0,Orphanet classification of rare cardiac diseases,5,Long chain 3-hydroxyacyl-CoA dehydrogenase def...
1,150.0,Orphanet classification of rare inborn errors ...,6,3-methylcrotonyl-CoA carboxylase deficiency
2,147.0,Orphanet classification of rare developmental ...,7,3C syndrome
3,147.0,Orphanet classification of rare developmental ...,8,"47,XYY syndrome"
4,147.0,Orphanet classification of rare developmental ...,9,Tetrasomy X syndrome
...,...,...,...,...
11099,147.0,Orphanet classification of rare developmental ...,689408,Shashi-Pena syndrome
11100,147.0,Orphanet classification of rare developmental ...,689422,Okur-Chung neurodevelopmental syndrome
11101,197.0,Orphanet classification of rare odontological ...,689430,Adenoid ameloblastoma
11102,147.0,Orphanet classification of rare developmental ...,689822,Structural heart defects-renal anomalies syndrome


In [62]:
df_enfermedades_clasificacion_gen = df_enfermedades_clasificacion.merge(orphadata['df_disease_gene'], on='ORPHAcode', how='left')
df_enfermedades_clasificacion_gen.drop(columns=['Disorder name', 'Gene synonyms'], inplace=True)
df_enfermedades_clasificacion_gen.info()
df_enfermedades_clasificacion_gen

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15308 entries, 0 to 15307
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   hchId              13877 non-null  float64
 1   hchTag             13877 non-null  object 
 2   ORPHAcode          15308 non-null  int64  
 3   preferredTerm      13877 non-null  object 
 4   Disorder type      8193 non-null   object 
 5   Gene symbol        8193 non-null   object 
 6   Gene name          8193 non-null   object 
 7   Gene type          8193 non-null   object 
 8   Locus              8191 non-null   object 
 9   Validation source  7441 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.2+ MB


,hchId,hchTag,ORPHAcode,preferredTerm,Disorder type,Gene symbol,Gene name,Gene type,Locus,Validation source
0,146.0,Orphanet classification of rare cardiac diseases,5,Long chain 3-hydroxyacyl-CoA dehydrogenase def...,Disease,HADHA,hydroxyacyl-CoA dehydrogenase trifunctional mu...,gene with protein product,"[{'GeneLocus': '2p23.3', 'LocusKey': 1}]",NaN
1,150.0,Orphanet classification of rare inborn errors ...,6,3-methylcrotonyl-CoA carboxylase deficiency,Disease,MCCC1,methylcrotonyl-CoA carboxylase subunit 1,gene with protein product,"[{'GeneLocus': '3q27.1', 'LocusKey': 1}]",22642865[PMID]_22264772[PMID]
2,150.0,Orphanet classification of rare inborn errors ...,6,3-methylcrotonyl-CoA carboxylase deficiency,Disease,MCCC2,methylcrotonyl-CoA carboxylase subunit 2,gene with protein product,"[{'GeneLocus': '5q13.2', 'LocusKey': 1}]",22642865[PMID]_22264772[PMID]
3,147.0,Orphanet classification of rare developmental ...,7,3C syndrome,Malformation syndrome,VPS35L,VPS35 endosomal protein sorting factor like,gene with protein product,"[{'GeneLocus': '16p12.3', 'LocusKey': 1}]",31712251[PMID]
4,147.0,Orphanet classification of rare developmental ...,7,3C syndrome,Malformation syndrome,DPYSL5,dihydropyrimidinase like 5,gene with protein product,"[{'GeneLocus': '2p23.3', 'LocusKey': 1}]",33894126[PMID]
...,...,...,...,...,...,...,...,...,...,...
15303,147.0,Orphanet classification of rare developmental ...,689408,Shashi-Pena syndrome,Malformation syndrome,ASXL2,ASXL transcriptional regulator 2,gene with protein product,"[{'GeneLocus': '2p23.3', 'LocusKey': 1}]",27693232[PMID]
15304,147.0,Orphanet classification of rare developmental ...,689422,Okur-Chung neurodevelopmental syndrome,NaN,NaN,NaN,NaN,NaN,NaN
15305,197.0,Orphanet classification of rare odontological ...,689430,Adenoid ameloblastoma,Disease,CTNNB1,catenin beta 1,gene with protein product,"[{'GeneLocus': '3p22.1', 'LocusKey': 1}]",35840721[PMID]
15306,147.0,Orphanet classification of rare developmental ...,689822,Structural heart defects-renal anomalies syndrome,NaN,NaN,NaN,NaN,NaN,NaN
